<a href="https://colab.research.google.com/github/BlancaCC/ProcesamientoDeDatosAGranEscala/blob/spark/SparkColabEjercicios2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark en Google Colab

Instalacion Octubre/2022

1.   Instalacion Java
2.   Instalacion de Spark
3.   Instalar PySpark

## De forma General para usar pyspark en Colab (2022) siga los siguientes pasos en una celda en Colab:

In [ ]:
# instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
# Descargar la ultima versión de java ( comprobar que existen los path de descarga)
# Download latest release. Update if necessary

!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
%ls -la /content/



In [ ]:
!tar xf spark-3.3.0-bin-hadoop3.tgz

In [ ]:
%ls -la

In [ ]:
# instalar pyspark
!pip install -q pyspark

# Variables de entorno

In [ ]:
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

# Iniciar la sesión de Spark con pyspark en el sistema

In [ ]:
from pyspark.sql import SparkSession

APP_NAME = "PDGE-tutorialSpark1"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
spark

In [ ]:
sc = spark.sparkContext
#obtener el contexto de ejecución de Spark del Driver.
sc

In [ ]:
array = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)
array

In [ ]:
print(array.collect())

In [ ]:
print(array.count())


In [ ]:
num3 = array.map(lambda elemento: 3*elemento)

In [ ]:
print(num3.collect())

# Ejemplos de operaciones con RDDs de Spark

In [ ]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)

print(numeros.reduce(lambda e1,e2: e1+e2))

In [ ]:
pnumeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = pnumeros.map(lambda e: 2*e)

print(rdd.collect())


## Pregunta TS1.1 ¿Cómo hacer para obtener una lista de los elementos al cuadrado?

In [ ]:
#¿Cómo hacer para obtener una lista de los elementos al cuadrado?
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

#rdd = numeros...
rdd = numeros.map(lambda e: e**2)

print(rdd.collect())


## Pregunta TS1.2 ¿Cómo filtrar los impares?

In [ ]:
#¿Cómo filtrar los impares?
#rddi = numeros.
rddi = numeros.filter(lambda n: n%2 == 0) # Mostrará los números impares
print(rddi.collect())


# Ejemplos de operaciones con números

Sumar n numeros

In [ ]:
rddnum=sc.range(1000000)
rddnum.reduce(lambda a,b: a+b)

In [ ]:
numeros = sc.parallelize([1,1,2,2,5])

unicos = numeros.distinct()

print (unicos.collect())

In [ ]:
numeros = sc.parallelize([1,2,3,4,5])

rdd = numeros.flatMap(lambda elemento: [elemento, 10*elemento])

print (rdd.collect())

Muestreo

In [ ]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = numeros.sample(True, 1.0)
# True = sin reemplazar
# 1.0 = fracción de elementos = 100%
print (rdd.collect())

Union 

In [ ]:
pares = sc.parallelize([2,4,6,8,10])
impares = sc.parallelize([1,3,5,7,9])

numeros = pares.union(impares)

print (numeros.collect())

reducción


In [ ]:
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

## Pregunta TS1.3 ¿Tiene sentido cambiar la suma por una diferencia? ¿Si se repite se obtiene siempre el mismo resultado?

In [ ]:
#Tiene sentido la reducción(elem1-elem2)?
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

# Respuesta TS1.3 

Tal operación **no** tiene sentido ya que la operación obtenida no es la esperada. 

Para entender qué es lo que está ocurriendo es necesario consultar la documentación oficial de spark [última consulta 30 de septiembre del 2022](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

al realizar parallelize por defecto se hacen dos particiones de RRD, por tanto al hacer el reduce se calcularía la operación en las respectivas dos particiones y tras esto se haría la resta de los resultados de ambas. 

Si quisiéramos que el resultado fuera el esperado deberiamos de especificar de que solo hubiera una partición (ver la celda siguiente)


In [ ]:
# Celda de ejemplo de cómo habría que programarlo para que el resultado fuera el correcto: 

#Tiene sentido la reducción(elem1-elem2)?
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10],1)

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

Como es de esperar si variamos el número de particiones la salida se verá afectada. Esto nos hace pensar que para poder paralelizar este tipo de operaciones es necesario que las operaciones sean **asociativas y conmutativas**. 

In [ ]:
#Tiene sentido la reducción(elem1-elem2) si la repetimos y los elementos están en otro orden?
numeros = sc.parallelize([2,1,4,3,5,6,7,8,9,10])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

In [ ]:
#Tiene sentido la reducción(elem1-elem2)? ¿qué pasa si cambiamos la paralelización a 5? ¿y si es 2?
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10],5)

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

#Acciones

In [ ]:
numeros = sc.parallelize([5,3,1,2,4])

print (numeros.take(3))
#¿Qué sucede si ponemos 30 en vez de 3 elementos?

### Solución 

Tomará los 30 primeros, en nuestro caso, puesto que solo hay 5 tomará los 5 prieros. 

In [ ]:
numeros = sc.parallelize([3,2,1,4,5])
# (La segunda función se utiliza como índice de ordenación)
print (numeros.takeOrdered(3, lambda elem: -elem))
# La función lambda se está utilizando para crear el índice de la lista de ordenación 

## Pregunta TS1.4 ¿Cómo lo ordenarías para que primero aparezcan los impares y luego los pares?

In [ ]:
#¿Cómo lo ordenarías para que primero aparezcan los impares y luego los pares?
numeros = sc.parallelize([3,2,1,4,5])
#print(numeros...
print (numeros.takeOrdered(5, lambda elem: -(elem %2)*elem))


# RDDs con string

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.collect())

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_long = palabras.map(lambda elemento: len(elemento))

print (pal_long.collect())

In [ ]:
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])

errors = log.filter(lambda elemento: elemento[0]=='E')

print (errors.collect())

In [ ]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras = lineas.flatMap(lambda elemento: elemento.split())

print (palabras.collect())

In [ ]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras_flat = lineas.flatMap(lambda elemento: elemento.split())
palabras_map = lineas.map(lambda elemento: elemento.split())

print (palabras_flat.collect())
print (palabras_map.collect())


## Pregunta TS1.5 ¿Cuántos elementos tiene cada rdd? ¿Cuál tiene más?


In [ ]:
#¿Cuántos elementos tiene cada rdd? ¿Cuál tiene más?
print (palabras_flat.collect())
print (palabras_map.collect())

### Respuesta TS1.5
El primero posee 6, mientras que el otro 4. Es evidente que el primero posee más. 

## Pregunta TS1.6 ¿De qué tipo son los elementos del rdd palabras_map? ¿Por qué palabras_map tiene el primer elemento vacío?

### Respuesta TS1.6

Son listas. 

Está vacía porque se generado a partir de un string sin ningún carácter ni espacio, que tras ser *separados* por la función  `split` se traduce en la lista vacía. 


In [ ]:
palabras_flat.take(1)

In [ ]:
palabras_map.take(1)


## Pregunta TS1.7. Prueba la transformación distinct si lo aplicamos a cadenas.

In [ ]:
# Prueba la transformación distinct si lo aplicamos a cadenas.¿Que realiza?
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])
print( log.distinct().collect() )

### Respuesta TS1.7 

Podemos observar que muestra los elementos de la lista sin repeticiones. 

## Pregunta TS1.8 ¿Cómo se podría obtener la misma salida pero utilizando una sola transformación y sin realizar la unión?

In [ ]:
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])

infos = log.filter(lambda elemento: elemento[0]=='I')
errors = log.filter(lambda elemento: elemento[0]=='E')

inferr = infos.union(errors)

print (inferr.collect())

### Respuesta TS1.8 

Para ello vamos a realizar directamente el filtro donde se admitan ambas condiciones.  

In [ ]:
#¿Cómo se podría obtener la misma salida pero utilizando una sola transformación y sin realizar la unión?
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])
result = log.filter(lambda elemento: elemento[0]=='I' or elemento[0]=='E')
print(result.collect())


# Diferencias entre hacer map y recuce con números y cadenas

In [ ]:
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

In [ ]:
#Tiene sentido esta operación?¿Cómo sale el resultado? pruebe los resultados
# empezando con 2 elementos e ir incrementando hasta tener 5

numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem2-elem1))

No tiene sentido porque el resultado no es correcto. 

In [ ]:
for i in range(2,6):
  print(f"Para {i} elementos, la lista que se  es {list(range(1,i+1))}")
  numeros = sc.parallelize(list(range(1,i+1)))
  print (f"Para {i} elementos", numeros.reduce(lambda elem1,elem2: elem2-elem1))

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.reduce(lambda elem1,elem2: elem1+"-"+elem2))
#y esta tiene sentido esta operación?
#Sí tiene 
# Qué pasa si ponemos elem2+"-"+elem1
#

In [ ]:
print (pal_minus.reduce(lambda elem1,elem2: elem2+"-"+elem1))

En este ejemplo se pone de manifiesto claramente lo que se comentó en la respuesta [Respuesta TS1.3](#respuesta-ts1.3). 

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr.collect())

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:v1)
print (rr2.collect())


## Pregunta TS1.9 ¿Cómo explica el funcionamiento de las celdas anteriores?

In [ ]:
#palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])
#print (pal_minus.reduce(lambda elem1,elem2: elem1+"-"+elem2))
#y esta tiene sentido esta operación?
# Qué pasa si ponemos elem2+"-"+elem1

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr.collect())


De cada dupla suma el valor de los elementos que comparten la misma clave. 

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:v1)
print (rr2.collect())


# TS1.10 Responda a las preguntas planteadas al hacer los cambios sugeridos en las siguiente celdas

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:'hola')
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:'hola')
print (rr2.collect())

In [ ]:
r = sc.parallelize([('A', 1),('C', 2),('A', 3),('B', 4),('B', 5)])
rr = r.groupByKey()
res= rr.collect()
for k,v in res:
    print (k, list(v))
# Que operación se puede realizar al RDD rr para que la operacion sea como un reduceByKey
#¿Y simular un group con un reduceByKey y un map?

### Respuestas TS1.10 

#### Transformar groupByKey para que se transforme como un reduce 




In [ ]:
from functools import reduce 
r = sc.parallelize([('A', 1),('C', 2),('A', 3),('B', 4),('B', 5)])
# Transformación con sentencias de python 
rr = (r.groupByKey()).collect()
res = list(map( lambda v: (v[0], reduce( lambda x,y: "hola", list(v[1]))), rr))
print(res)

### Simular un GroupBy con un reduceByKey y un map

De cada par clave valor la idea es introducir cada valor a una lista y despues realizar `reduceByKey` donde la operación de *reduce* es concaternar las listas. 

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
r_map = r.map(lambda x: (x[0],[x[1]]))
rr1 = r_map.reduceByKey(lambda v1,v2: v1+v2)
print (rr1.collect())

In [ ]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('B',5),('C',6)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
# Prueba a cambiar las claves del rdd1 y rdd2 para ver cuántos elementos se crean

In [ ]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
#Modifica join por leftOuterJoin, rightOuterJoin y fullOuterJoin ¿Qué sucede? 


In [ ]:
rdd = sc.parallelize([('A',1),('B',2),('C',3),('A',4),('A',5),('B',6)])

res = rdd.sortByKey(False)

print (res.collect())

# Utilización de ficheros

In [ ]:

# Crea una lista de 1000 números y la guarda.
# Da error si 'salida' existe, descomenta la linea de borrar en ese caso.
%rm -rf salida
numeros = sc.parallelize(range(0,1000))
numeros.saveAsTextFile('salida')


In [ ]:
ls -la

In [ ]:
%ls -la salida/*


In [ ]:
#%cat salida/part-00000

In [ ]:
# Recupera el fichero guardado y realiza la suma
n2 = sc.textFile('salida').map(lambda a:int(a))
print(n2.reduce(lambda v1,v2: v1 + v2))

# Prueba este código y mira qué genera?
# Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
#    (pe c.parallelize(xrange(0,1000),8))


## Pregunta TS1.11 Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
  (pe c.parallelize(xrange(0,1000),8))


In [ ]:
# Recupera el fichero guardado y realiza la suma
n2 = sc.textFile('salida').map(lambda a:int(a))
print(n2.reduce(lambda v1,v2: v1 + v2))

# Prueba este código y mira qué genera?
# Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
sc.parallelize(range(0,1000),8)
#numeros.saveAsTextFile('salida3')


In [ ]:
# Prueba este código y mira qué genera?
# Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
%rm -rf salida3
n = sc.parallelize(range(0,17),8)
n.saveAsTextFile('salida3')
%cat salida3/part-00000


### Respuesta TS1.11
Podemos observar que cada proceso en paralelo sobreescribe el fichero y por tanto solo se refleja la última que ha sido guardada. 

In [ ]:
%cat salida3/part-00000

In [ ]:
%rm -rf salida
%ls -la


# El quijote

Montar el directorio de trabajo utilizando Google Drive
Loading Your Data into Google Colaboratory.
1. First of all, Upload your Data to your Google Drive.
2. Run the following script in colab shell.
3. Copy the authorization code of your account.
4. Paste the authorization code into the output shell.
5. Congrats! Now your Google Drive is mounted to this location /content/gdrive/MyDrive/




In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

In [ ]:
%ls /content/gdrive/MyDrive/

In [ ]:
#!cp "/content/gdrive/My Drive/quijote.txt" .
# Nota: He tenido que añadir la ruta esta para concreta para que lo lea en mi drive
!cp "./gdrive/MyDrive/Colab\ Notebooks/quijote.txt" .

In [ ]:
%ls -la

In [ ]:
%pwd



# Procesando el QUIJOTE

In [ ]:
# Nota: He tenido que añadir la ruta esta para concreta para que lo lea en mi drive
quijote = sc.textFile("gdrive/MyDrive/Colab\ Notebooks/quijote.txt")
quijote.take(10)

Transformaciones

In [ ]:
charsPerLine = quijote.map(lambda s: len(s))
allWords = quijote.flatMap(lambda s: s.split())
allWordsNoArticles = allWords.filter(lambda a: a.lower() not in ["el", "la"])
allWordsUnique = allWords.map(lambda s: s.lower()).distinct()
sampleWords = allWords.sample(withReplacement=True, fraction=0.2, seed=666)
weirdSampling = sampleWords.union(allWordsNoArticles.sample(False, fraction=0.3))
# cómo funciona cada transformación

- `charsPerLine`: Caracteres por líneas. 
- `allWords`: Contiene un RRD cuyos elementos son variables de tipo `String` con las palabras del quijote.
- `allWordsNoArticles`: Minimiza las palabras del RDD `allWords` y devuelve todas menos los artículos `el` y `la`.
- `allWordsUnique`:  Minimiza las palabras del RDD `allWords` y deja solo un representante. 
- `sampleWords`: Toma una muestra aleatora de tamaño aproximadamente $20\%$  del RRD de `allWords` con remplazamiento y semilla $666$. 
- `weirdSampling`: Devuelve un RDD que contiene los elementos de `sampleWords` y una mueestra aleatoria de `allWordsNoArticles` sin remplazamiento y de tamaño alrededor del $30\%$.

Se detallarán las funciones en la respuesta a TS2.1. 


In [ ]:
allWordsUnique.take(10)

## Pregunta TS2.1 Explica la utilidad de cada transformación y detalle para cada una de ellas si cambia el número de elementos en el RDD resultante. Es decir si el RDD de partida tiene N elementos, y el de salida M elementos, indica si N>M, N=M o N<M.

-  map
- flatmap
- filter
- distinct
- sample
- union


## Respuesta TS2.1

### Map

Los argumentos de `map` son `map(f) ` donde `f` a cuyo dominio pertenecen los elementos del RDD. 

El RDD resultante son las respectivas imágenes de aplicar los elementos del RDD de entrada a `f`. 

Es por ello que N = M.

Ejemplo: 


In [ ]:
numeros = sc.parallelize(list(range(10)))
rdd = numeros.map(lambda x: x*10+x)
print(rdd.collect())

### Flatmap
Funciona como un `map` el cual generaría un RDD cuyos elementos deben ser iterables, a éstos se le aplica la operación de *flattening*, es decir *saca* los elementos que estén en una lista (solo actúa a un nivel, una lista de lista devolvería una lista). 

No se puede establecer que ninguna relación entre N y M ya que si hay listas  vacías entonces pudiera darse el caso que N > M (ver ejemplo 2 que se mostrará a continuación). Aunque si imponemos que no hay listas vacías entonces $N \leq M$. 



In [ ]:
# Ejemplo 1: N < M
numeros = sc.parallelize(list(range(10)))
rdd = numeros.flatMap(lambda x: [x,x*10+x])
print(rdd.collect())

# Ejemplo 2: N > M
def lista_recursiva (n):
  if n <= 1:
    return []
  else:
    return [lista_recursiva(n-1)]
numeros = sc.parallelize(list(range(1,4)))
print("Ejemplo 2: si se aplicara un map solo  solo:\n",
      numeros.map(lambda x: lista_recursiva(x)).collect())
print("Ejemplo 2: de qué habría en un flapMap:\n",
      numeros.flatMap(lambda x: lista_recursiva(x)).collect())

# Ejemplo 3: N = M
def lista_recursiva_2 (n):
  if n <= 1:
    return [n]
  else:
    return [lista_recursiva_2(n-1)]

numeros = sc.parallelize(list(range(3)))
print("Ejemplo 3: si se aplicara un map solo:\n",
      numeros.map(lambda x: lista_recursiva_2(x)).collect())

print("Ejemplo 3: si se aplicara un flapMap:\n",
      numeros.flatMap(lambda x: lista_recursiva_2(x)).collect())



## Filter 

Recive una función con dominio en los elemntos del RDD de partida y salida un booleano. 
Devuelve un RDD con los elementos cuya imagen sea `True`.
En este caso $N \geq M$, donde la igualdad se dará si todos los elementos tienen imagen `True`.

In [ ]:
numeros = sc.parallelize([0,1,2,3,4])
rddi = numeros.filter(lambda x: x < 2) # Mostrará los números menores de dos
print(rddi.collect())

## Distinct 

Elimina los elementos repetidos dejando solo una incidencia de ellos. 
Por tanto $N \geq M$ donde la igualdad se da si no hay ningún elemento repetido. 


In [ ]:
# Ejemplo sencillo
n = sc.parallelize([0,0,1,1,0,2])
rdd = n.distinct()
print(rdd.collect())

# Ejemplo complejo, compara elemento a elemento
n = sc.parallelize([('a',2),('a',2), ('a',1), ('b',1),])
rdd = n.distinct()
print(rdd.collect())

## Sample

Es un mecanismo para extraes muestras aleatorias, sus argumentos son: 
`sample(withReplacement, fraction, seed=None`
- `withReplacement`: booleano que indica si hay reemplazamiento. 
- `fraction`: decimal en $[0,1]$ fracción de la mustra que se desea. **No se garantiza que la muestra siempre de esa fracción, puede ser menor o mayor**.
- `seed` semilla. 

Por la observación de que no se garantice la fracción no se puede establecer ninguna relación de orden entre $N$ y $M$, mostramos tales situaciones a continuación. 

In [ ]:
# Ejemplo 1: N > M
print(f'RDD de partida {list(range(5))} N = 5')
print('Ejemplo 1: N > M')
numeros = sc.parallelize(list(range(5)))
rdd = numeros.sample(True, 0.5, 0)
print(rdd.collect())
# Ejemplo 2: N < M
print('Ejemplo 2: N < M')
numeros = sc.parallelize(list(range(5)))
rdd = numeros.sample(True, 1,1)
print(rdd.collect())
# Ejemplo 3: N = M
print('Ejemplo 3: N = M')
numeros = sc.parallelize(list(range(5)))
rdd = numeros.sample(False, 1,1)
print(rdd.collect())

## Union

Dados dos RDD devuelve la unión de los dos, es decir, uno RDD con todos los elementos de ambos (cabe destacar que en caso de estar repetidos los elementos los volvería a repetir). 
Para este si $N_1, N_2$ son los respectivos tamaños de los RDD de entrada y $M$ el de salida se cumple que
$N_1 + N_2 = M$

In [ ]:
# Ejemplos 1: 
n1 = sc.parallelize([1,2,3])
n2 = sc.parallelize([1,3,5])
m = n1.union(n2)
print (m.collect())

# Ejemplo 2: Une elementos de distinto tipo
n1 = sc.parallelize([1,2,3])
n2 = sc.parallelize(['1','3','5'])
m = n1.union(n2)
print (m.collect())

In [ ]:
numLines = quijote.count()
numChars = charsPerLine.reduce(lambda a,b: a+b) # also charsPerLine.sum()
sortedWordsByLength = allWordsNoArticles.takeOrdered(20, key=lambda x: -len(x))
numLines, numChars, sortedWordsByLength

## Pregunta TS2.2 Explica el funcionamiento de cada acción anterior 





- Tengamos presente que `quijote` es un RDD donde cada elemento es una línea del quijote. 
Por lo que  `quijote.count()` dirá el número de líneas del quijote. 

- La variable `charsPerLine` contiene el número de caracteres que hay en cada línea, luego
`numChars = charsPerLine.reduce(lambda a,b: a+b) # also charsPerLine.sum(` devuelve el número de caracteres total. 

- La variable `allWordsNoArticles` continee las palabras minimizadas con un único representante. 
por lo que 
```python
sortedWordsByLength = allWordsNoArticles.takeOrdered(20, key=lambda x: -len(x))
```
Toma las 20 primeras ordenadas de mayor tamaño a menor. 



## Pregunta: Implementa la opción count de otra manera:


*   utilizando transformaciones map y reduce
*   utilizando solo reduce en caso de que sea posible.

### Respuesta 

#### Utilizando transformaciones map y reduce

Cada línea se mapeará por un 1 y con reduce se sumarán. 

In [ ]:
lineas = quijote.map(lambda linea : 1)
suma_lineas = lineas.reduce(lambda x,y: x+y)
print(suma_lineas)

### Utilizando solo reduce 

La idea clave está en la función del reduce, que debe de ser capaz de tener como entrada cadenas de caracteres y números. 

Si se encuentra con una cadena de caracter es porque se trata de una línea que debe de ser contada, por tanto la cambia a 1 y la suma. 

Si se encuentra un número es de la cuenta anteriores y por tanto lo suma tal cual. 

In [ ]:
def reduce_suma_lineas(x,y):
  if type(x) == str:
    x = 1
  if type(y) == str:
    y = 1
  return x+y

suma_lineas = quijote.reduce(reduce_suma_lineas)
print(suma_lineas)

# Operaciones K-V (Clave Valor)

In [ ]:
import requests
import re
allWords = allWords.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.lower()).split(" ")).filter(lambda a: len(a)>0)
allWords2 = sc.parallelize(requests.get("https://gist.githubusercontent.com/jsdario/9d871ed773c81bf217f57d1db2d2503f/raw/585de69b0631c805dabc6280506717943b82ba4a/el_quijote_ii.txt").iter_lines())
allWords2 = allWords2.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.decode("utf8").lower()).split(" ")).filter(lambda a: len(a)>0)

In [ ]:
allWords.take(10)

In [ ]:
allWords2.take(10)

In [ ]:
words = allWords.map(lambda e: (e,1))
words2 = allWords2.map(lambda e: (e,1))

words.take(10)

In [ ]:
frequencies = words.reduceByKey(lambda a,b: a+b)
frequencies2 = words2.reduceByKey(lambda a,b: a+b)
frequencies.takeOrdered(10, key=lambda a: -a[1])

In [ ]:
res = words.groupByKey().takeOrdered(10, key=lambda a: -len(a))
res # To see the content, res[i][1].data
#res[i][1].data

In [ ]:
joinFreq = frequencies.join(frequencies2)
joinFreq.take(10)

In [ ]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

## Pregunta TS2.3 Explica el proposito de cada una de las operaciones anteriores

Las operaciones en cadena son: 

1. `. joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1])))`
Devuelve una tupla con la palabra y la diferencia entre el números de operaciones en la primera parte del Quijote menos las apariciones en la segunda parte entre la suma total de las apariciones en los dos libros. 

La idea que subyace en el cálculo es ver si la frecuencia de aparición de palabras es la misma. Será próxima a cero si es así, cercana a menos uno si aparece mayoritariamente el el segundo y cercana a uno si lo es en primero. 

2. `takeOrdered(10, lambda v: -v[1])`
Ordena de mayor a menor las frecuencias optenidas. 


3. `joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))`

Repite el paso 1. 

4. `takeOrdered(10, lambda v: +v[1])`
Ahora ordena de de menor a mayor las frecuencias. 

## Pregunta TS2.4 ¿Cómo puede implementarse la frecuencia con groupByKey y transformaciones?

In [ ]:
frequencies_group = words.groupByKey().map(lambda e: (e[0],sum(list(e[1]))))
frequencies_group.take(10)

# Optimizaciones


## Pregunta TS2.5 ¿Cuál de las dos siguientes celdas es más eficiente? Justifique la respuesta.

La más eficiente es la segunda, puesto que se está guardando en caché los datos y la operación de cálculo de apariciones no tiene que repetirse. 

In [ ]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

In [ ]:
result = joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1])))
result.cache()
result.takeOrdered(10, lambda v: -v[1]), result.takeOrdered(10, lambda v: +v[1])

In [ ]:
result.coalesce(numPartitions=2) # Avoids the data movement, so it tries to balance inside each machine
result.repartition(numPartitions=2) # We don't care about data movement, this balance the whole thing to ensure all machines are used

In [ ]:
result.take(10)
allWords.cache() # allWords RDD must  stay in memory after computation, we made a checkpoint (well, it's a best effort, so must might be too strong)
result.take(10)

In [ ]:
from pyspark import StorageLevel
# https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#rdd-persistence
allWords2.persist(StorageLevel.MEMORY_AND_DISK) # Now it will be preserved on disk also

In [ ]:
!rm -rf palabras_parte2
allWords2.saveAsTextFile("palabras_parte2")

## Pregunta TS2.6 Antes de guardar el fichero, utilice coalesce con diferente valores ¿Cuál es la diferencia?

De acorde a la documentación de [coalescence](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.coalesce.html)

In [ ]:
allWords2.getNumPartitions()

In [ ]:
for j in range(1,10):
  print("numero de particiones = {}".format(j))
  !rm -rf words_coalesce2
  allWords = allWords2.coalesce(numPartitions=j).saveAsTextFile("words_coalesce2")
  !ls words_coalesce2



Se observa que el número de particiones de disco se estanca cuando el número de particiones llega a 2, dejando así de incrementar su valor para las sucesivas. Cabe destacar, que sería equivalente el usar la función repartition, ya que no se tiene en cuenta que los datos se muevan en el disco para relizar dichas particiones.